<a href="https://colab.research.google.com/github/nuyhc/RhythmStudy/blob/main/1.%20PyTorch/PTM_5_%EB%B0%95%EC%A7%80%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 엔드투엔드 

* 10장 => 데이터를 읽는 - 1단계
    * .MHD + .RAW
* 11, 12장 => 결절을 분류하는 문제에 집중 - 4단계
* 13장 => 다시 2단계(결절 후보들을 찾아 세그멘테이션 하기) - 2단계
* 14장 => 3단계(그룹화)와 5단계(결절 분석 및 진단) - 3단계, 5단계

데이터 로딩 -> 세그멘테이션 -> 그룹화 -> 분류 -> 결절 분석 및 진단

[LUNA 데이터 사이트] https://luna16.grand-challenge.org/

# [PTM] 10장. 여러 데이터 소스를 통합 데이터셋으로 합치기

* 원본 데이터 파일을 읽어 처리하기
* 데이터를 표현하는 파이썬 클래스 구현
* 데이터를 파이토치에서 사용 가능한 포맷으로 변환
* 훈련 데이터와 검증 데이터의 시각화

In [19]:
!git clone https://github.com/deep-learning-with-pytorch/dlwpt-code

Cloning into 'dlwpt-code'...
remote: Enumerating objects: 703, done.
remote: Total 703 (delta 0), reused 0 (delta 0), pack-reused 703
Receiving objects: 100% (703/703), 176.00 MiB | 17.51 MiB/s, done.
Resolving deltas: 100% (309/309), done.
Checking out files: 100% (228/228), done.


## 10.1 원본 CT 데이터 파일

CT 데이터
* .mhd 파일: 메타데이터 헤더 정보 포함 파일
* .raw 파일: 3차원 배열을 만들 원본 데이터 바이트를 포함 파일  
각 파일 이름은 **시리즈 UID**(DICOM Digital Imaging and Communications in Medicals  명명법에 유래했다)라고 불리는 CT 스캔 단일 식별자로 시작한다.  
예를 들어 시리즈 UID 1.2.3의 경우, 1.2.3.mhd와 1.2.3.raw의 두 가지 파일이 있다.  

`일단, CT 데이터에 적용할 좌표계 변환이 있어야 한다 정도만 기억해두자`

LUNA
* 애노테이션 데이터: 각 결절의 좌표 목록, 악성 여부, 해당 CT 스캔의 시리즈 UID
    * 결절 좌표가 좌표계 변환 정보를 거치면 결절의 중심에 해당하는 복셀의 인덱스, 행, 열 정보가 생김

(I,R,C)좌표를 사용 -> CT 데이터의 작은 3차원 부분 단면을 얻어 모델에 대한 입력으로 사용 -> 3차원 배열과 함게, 훈련 샘플 튜플의 나머지를 구성해야 함 (샘플 배열, 결절의 상태 플래그, 시리즈 UID, 결절 후보군의 CT 리스트 중 이 샘플이 몇 번째 인덱스인지 등이 포함)  
`파이토치가 Dataset 서브클래스를 통해 얻고자 하는 것이 정확히 이 튜플이다. 동시에 이 튜플을 원본 데이터를 표준 구조의 파이토치 텐서로 변환하는 과정의 마지막 부분이다.`

## 10.2 LUNA 애노테이션 데이터 파싱
LUNA에서 제공하는 csv 파일을 먼저 파싱하여 각 CT 스캔 중 관심 있는 부분을 파악해볼 필요가 있음

얻을 수 있는 정보
* 좌표 정보
* 해당 좌표 지점이 결절인지 여부
* CT 스캔에 대한 고유 식별자

[LUNA 데이터 사이트] https://luna16.grand-challenge.org/

candidates.csv 파일에는 조직 덩어리가 결절일 가능성이 있는지와 악성 종양 또는 양성 종양 여부, 그리고 그 밖에 정보가 들어있다. 추후 훈련, 검증 데이터셋으로 나눌 것이다.

In [5]:
# 파일의 행 개수 카운트
!wc -l /content/drive/MyDrive/리듬스터디/data/LUNA/candidates.csv

551066 /content/drive/MyDrive/리듬스터디/data/LUNA/candidates.csv


In [6]:
# 파일 앞부분 일부를 출력, 첫 줄은 칼럼 헤더
# .csv 파일의 첫 행은 열 헤어를 정의함
!head /content/drive/MyDrive/리듬스터디/data/LUNA/candidates.csv

seriesuid,coordX,coordY,coordZ,class
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-56.08,-67.85,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,53.21,-244.41,-245.17,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.66,-121.8,-286.62,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-33.66,-72.75,-308.41,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-32.25,-85.36,-362.51,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-26.65,-203.07,-165.07,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-74.99,-114.79,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-16.14,-248.61,-239.55,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,135.89,-141.41,-252.2,0


??? 리눅스 grep 명령어

In [18]:
# 결절이라서 1로 끝나는 행 개수 카운트
!grep ',1$' /content/drive/MyDrive/리듬스터디/data/LUNA/candidates.csv | wc -l

0


annotations.csv 파일에는 결절로 플래그된 후보들에 대한 정보가 포함되어 있다. 특별히 `diameter_mm` 정보를 주목할 만하다. 대략 1,200개 결절에 대한 크기 정보가 있다. 이 유용한 정보를 결절 크기의 분포로 가정하여 훈련, 검증 데이터를 만드는 데 유용하게 사용할 수 있다.

In [20]:
!wc -l /content/drive/MyDrive/리듬스터디/data/LUNA/annotations.csv

1187 /content/drive/MyDrive/리듬스터디/data/LUNA/annotations.csv


In [22]:
!head /content/drive/MyDrive/리듬스터디/data/LUNA/annotations.csv

seriesuid,coordX,coordY,coordZ,diameter_mm
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481
1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208,69.63901724,-140.9445859,876.3744957,5.786347814
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,-24.0138242,192.1024053,-391.0812764,8.143261683
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,2.441546798,172.4648812,-405.4937318,18.54514997
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,90.93171321,149.0272657,-426.5447146,18.20857028
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,89.54076865,196.4051593,-515.0733216,16.38127631
1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028192176989979435275,81.50964574,54.9572186,-150.3464233,10.36232088
1.3.6.1.4.1.14519.5.2.1.6279.6001.10268196240

### 10.2.1 훈련셋과 검증셋
supervised learning 작업은 데이터를 training set, validation set으로 나눈다.  
일단 크기 순으로 정렬한 후 매 N번째에 대해 검증셋을 넣어 분포를 반영한 검증셋을 구성하자.
* 안타깝게도 annotations.csv에서 제공하는 위치 정보는 candidates.csv의 좌표와 정확하게 일치하지는 않는다.

In [23]:
!grep 100225287222365663678666836860 /content/drive/MyDrive/리듬스터디/data/LUNA/annotations.csv

1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481


In [26]:
!grep '100225287222365663678666836860.*,1$' /content/drive/MyDrive/리듬스터디/data/LUNA/candidates.csv

### 10.2.2 애노테이션 데이터와 후보 데이터 합치기
annotations.csv와 candidates.csv의 정보를 합치는 `getCandidateInfoList` 함수를 만들어보자.  
각 결절 정보를 담아둘 `named tuple`을 파일 상단에 두고 사용하자

In [28]:
from collections import namedtuple

CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)

이 튜플은 우리가 필요로 하는 CT 데이터가 빠져 있으니 훈련 샘플이 아니다.  
이 데이터는 전문가의 애노테이션 데이터에 대해 나름대로 깔끔하게 다듬어진 통합 인터페이스를 나타낸다. 
* 모델 훈련 작업과 지저분한 데이터를 처리하는 작업을 분리하는 것이 매우 중요(그렇지 않으면 훈련 루프가 금세 지저분해짐)

`NoduleInfoTuple` 인스턴스 리스트를 만드는 함수는 in-memory caching decorator를 사용하고 디스크 파일 경로를 얻는다.

In [30]:
import copy
import csv
import functools
import glob
import os

@functools.lru_cache(1) # 표준 인메모리 캐싱 라이브러리
def getCandidateInfoList(requireOnDisk_bool=True):  # requireOnDisk_bool은 디스크에 없는 데이터는 걸러내기 위함
    mhd_list = glob.glob('data-unversioned/part2/luna/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

/bin/bash: -c: line 0: syntax error near unexpected token `1'
/bin/bash: -c: line 0: `functools.lru_cache(1)'


후보 정보를 얻었다면 annotations.csv의 직경 정보를 합치자.  
먼저 애노테이션 정보는 series_uid로 그룹화해서 두 파일에서 일치하는 행을 찾아내는 키로 사용하자

In [ ]:
    diameter_dict = {}
    with open('data/part2/luna/annotations.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])

            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )

그리고 이제 candidates.csv의 정보를 사용하여 전체 후보 리스트를 만들자

In [ ]:
    candidateInfo_list = []
    with open('data/part2/luna/candidates.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm = 0.0
            for annotation_tup in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                    if delta_mm > annotationDiameter_mm / 4:
                        break
                else:
                    candidateDiameter_mm = annotationDiameter_mm
                    break

            candidateInfo_list.append(CandidateInfoTuple(
                isNodule_bool,
                candidateDiameter_mm,
                series_uid,
                candidateCenter_xyz,
            ))

이제 데이터를 정렬 후 반환한다

In [ ]:
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

noduleInfo_list의 튜플 멤버 순서를 이 정렬로 만들어진다. 이렇게 데이터를 정렬하면 CT 단면들을 모아 결절 직경에 대해 잘 분포된 실제 결절을 반영하는 덩어리를 얻어올 수 있게 된다.

## 10.3 개별 CT 스캔 로딩
디스크에서 CT 데이터를 얻어와 파이썬 객체로 변환해서 3차원 결절 밀도 데이터로 사용할 수 있도록 만드는 작업이다. 결졀 애노테이션 정보는 원본 데이터에서 얻어내고자 하는 영역에 대한 map이라고 생각하면 된다. 
* CT 스캔을 읽어 복셀 배열을 만든 후 환자 좌표를 배열 인덱스로 변환하는 부분

CT 스캔 파일의 원래 포맷은 DICOM(www.dicomstandard.org) 이라고 부른다. DICOM 표준의 최초 버전은 1984년에 만들어졌다
 * LUNA는 우리가 사용하려는 데이터를 MetalO(https://itk.org/Wiki/MetalO/Documentation#Quick_Start) 포맷으로 변환해 놓았고 사용하기도 쉽다.
 * 데이터 파일 포맷을 블랙박스로 간주하고 친숙한 넘파이 배열로 읽어들이기 위해 SimpleITK를 사용할 것이다.

In [32]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 18.3 MB/s eta 0:00:00


In [ ]:
import SimpleITK as sitk
import numpy as np

class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(
            'data-unversioned/part2/luna/subset*/{}.mhd'.format(series_uid)
        )[0]

        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

### 10.3.1 하운스필드 단위

In [ ]:
ct_a.clip(-1000, 1000, ct_a)

In [ ]:
self.series_uid = series_uid
self.hu_a = ct_a

## 10.4 환자 좌표계를 사용해 결절 위치 정하기

### 10.4.1 환자 좌표계

### 10.4.2 CT 스캔 형태와 복셀 크기

### 10.4.3 밀리리터를 복셀 주소로 변환하기
1. 좌표를 XYZ 체계로 만들기 위해 IRC에서 CRI로 뒤집는다.
2. 인덱스를 복셀 크기로 확대축소한다.
3. 파이썬의 @를 사용하여 방향을 나타내는 행렬과 행렬곱을 수행한다.
4. 기준으로부터 오프셋을 더한다.

In [ ]:
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])

def irc2xyz(coord_irx, origin_xyz, vxSize_xyz, directions_a):
    cri_a = np.array(coord_irc)[::-1]. # 넘파이 배열로 변환하며 순서를 바꾼다.
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coords_xyz = (direction_a @ (cri_a * vxSize_xyz)) + origin_a  # 2,3,4단계를 한 줄로 실행한다.
    return XyzTuple(*coords_xyz)


def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a  # 4,3,2단계를 실행한다.
    cri_a = np.round(cri_a)  # 정수로 변환하기 전에 적절히 반올림해준다.
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))  # 섞으면서 정수로 변환한다.

In [ ]:
class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(
            'data-unversioned/part2/luna/subset*/{}.mhd'.format(series_uid)
        )[0]

        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

        ct_a.clip(-1000, 1000, ct_a)

        self.series_uid = series_uid
        self.hu_a = ct_a

        self.origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
        self.vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

### 10.4.4 CT 스캔에서 결절 추출하기

In [ ]:
    def getRawCandidate(self, center_xyz, width_irc):
        center_irc = xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a,
        )

        slice_list = []
        for axis, center_val in enumerate(center_irc):
            start_ndx = int(round(center_val - width_irc[axis]/2))
            end_ndx = int(start_ndx + width_irc[axis])

            assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, center_irc, axis])

            if start_ndx < 0:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                start_ndx = 0
                end_ndx = int(width_irc[axis])

            if end_ndx > self.hu_a.shape[axis]:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                end_ndx = self.hu_a.shape[axis]
                start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])

            slice_list.append(slice(start_ndx, end_ndx))

        ct_chunk = self.hu_a[tuple(slice_list)]

        return ct_chunk, center_irc

## 10.5 간단한 데이터셋 구현

In [ ]:
import torch
import torch.cuda
from torch.utils.data import Dataset

from util.disk import getCache
from util.util import XyzTuple, xyz2irc
from util.logconf import logging

In [ ]:
    def __len__(self):
        return len(self.candidateInfo_list)

    def __getitem__(self, ndx):
        return (
            candidate_t,
            pos_t,
            candidateInfo_tup.series_uid,
            torch.tensor(center_irc),
        )

In [ ]:
    def __getitem__(self, ndx):
        candidateInfo_tup = self.candidateInfo_list[ndx]
        width_irc = (32, 48, 48)

        candidate_a, center_irc = getCtRawCandidate(
            candidateInfo_tup.series_uid,
            candidateInfo_tup.center_xyz,
            width_irc,
        )

In [ ]:
        candidate_t = torch.from_numpy(candidate_a)
        candidate_t = candidate_t.to(torch.float32)
        candidate_t = candidate_t.unsqueeze(0)

In [ ]:
        pos_t = torch.tensor([
                not candidateInfo_tup.isNodule_bool,
                candidateInfo_tup.isNodule_bool
            ],
            dtype=torch.long,
        )

In [ ]:
LunaDataset()[0]

### 10.5.1 getCtRawCandidate 함수로 후보 배열 캐싱하기

In [ ]:
@functools.lru_cache(1, typed=True)
def getCt(series_uid):
    return Ct(series_uid)

@raw_cache.memoize(typed=True)
def getCtRawCandidate(series_uid, center_xyz, width_irc):
    ct = getCt(series_uid)
    ct_chunk, center_irc = ct.getRawCandidate(center_xyz, width_irc)
    return ct_chunk, center_irc

### 10.5.2 LunaDataset.__init__으로 데이터셋 만들기

In [ ]:
class LunaDataset(Dataset):
    def __init__(self,
                 val_stride=0,
                 isValSet_bool=None,
                 series_uid=None,
            ):
        self.candidateInfo_list = copy.copy(getCandidateInfoList())

        if series_uid:
            self.candidateInfo_list = [
                x for x in self.candidateInfo_list if x.series_uid == series_uid
            ]

### 10.5.3 훈련/검증 분리

In [ ]:
        if isValSet_bool:
            assert val_stride > 0, val_stride
            self.candidateInfo_list = self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
        elif val_stride > 0:
            del self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list

In [ ]:
from p2ch10.dsets import getCandidateInfoList, getCt, LunaDataset
candidateInfo_list = getCandidateInfoList(requireOnDisk_bool=False)
positiveInfo_list = [x for x in candidateInfo_list if x[0]]
diameter_list = [x[1] for x in positiveInfo_list]

### 10.5.4 데이터 렌더링

In [ ]:
%matplotlib inline
from p2ch10.vis import findPositiveSamples, showCandidate
positiveInfo_list = findPositiveSamples()

# [PTM] 11장. 종양 탐지를 위한 분류 모델 훈련

* 파이토치 DataLoader로 데이터 로딩하기
* CT 데이터에서 분류를 수행하는 모델 만들기
* 애플리케이션 기본 구조 설정
* 메트릭 로깅과 표시

## 11.1 기본 모델과 훈련 루프

구현의 기본 구조
* 모델을 초기화하고 데이터를 로딩한다.
* 어느 정도 임의로 선택한 에포크 수로 루프를 반복한다.
    * LunaDataset이 반환한 훈련 데이터의 배치 루프를 돈다.
    * 백그라운드에서 데이터로더 worker 프로세스는 적합한 배치를 읽어들인다.
    * batch를 분류 모델에 전달하여 결과를 얻는다.
    * 추정 결과를 실측 데이터와 비교하여 손실을 계산한다.
    * 임시 데이터 구조에 모델의 성능 메트릭을 기록한다.
    * 오차 역전파로 모델 가중치를 조정한다.
    * 검증 데이터 배치로 루프 반복
    * 검증 데이터 배치를 읽어들인다.
    * 배치를 분루하고 손실을 계산한다.
    * 모델이 검증 데이터에 대해 얼마나 잘 동작했는지를 기록한다.
    * 매 에포크마다 진행 상황과 성능 정보를 출력한다.

## 11.2 애플리케이션의 메인 진입점

In [ ]:
import datetime

from util.util import importstr
from util.logconf import logging
log = logging.getLogger('nb')

def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')  # 4코어 8스레드 CPU로 가정했는데 수정 가능
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))  # __import__보다 깔끔한 호출 방식
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}).main()".format(app, argv))

In [ ]:
if __name__ == '__main__':
    LunaTrainingApp().mane()

In [ ]:
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

        parser = argparse.ArgumentParser()
        parser.add_argument('--batch-size',
            help='Batch size to use for training',
            default=32,
            type=int,
        )
        parser.add_argument('--num-workers',
            help='Number of worker processes for background data loading',
            default=8,
            type=int,
        )
        parser.add_argument('--epochs',
            help='Number of epochs to train for',
            default=1,
            type=int,
        )
        parser.add_argument('--balanced',
            help="Balance the training data to half positive, half negative.",
            action='store_true',
            default=False,
        )
        parser.add_argument('--augmented',
            help="Augment the training data.",
            action='store_true',
            default=False,
        )
        parser.add_argument('--augment-flip',
            help="Augment the training data by randomly flipping the data left-right, up-down, and front-back.",
            action='store_true',
            default=False,
        )
        parser.add_argument('--augment-offset',
            help="Augment the training data by randomly offsetting the data slightly along the X and Y axes.",
            action='store_true',
            default=False,
        )
        parser.add_argument('--augment-scale',
            help="Augment the training data by randomly increasing or decreasing the size of the candidate.",
            action='store_true',
            default=False,
        )
        parser.add_argument('--augment-rotate',
            help="Augment the training data by randomly rotating the data around the head-foot axis.",
            action='store_true',
            default=False,
        )
        parser.add_argument('--augment-noise',
            help="Augment the training data by randomly adding noise to the data.",
            action='store_true',
            default=False,
        )

        parser.add_argument('--tb-prefix',
            default='p2ch12',
            help="Data prefix to use for Tensorboard run. Defaults to chapter.",
        )
        parser.add_argument('comment',
            help="Comment suffix for Tensorboard run.",
            nargs='?',
            default='dlwpt',
        )

        self.cli_args = parser.parse_args(sys_argv)
        self.time_str = datetime.datetime.now().strftime('%Y-%m-%d_%H.%M.%S')

        self.trn_writer = None
        self.val_writer = None
        self.totalTrainingSamples_count = 0

        self.augmentation_dict = {}
        if self.cli_args.augmented or self.cli_args.augment_flip:
            self.augmentation_dict['flip'] = True
        if self.cli_args.augmented or self.cli_args.augment_offset:
            self.augmentation_dict['offset'] = 0.1
        if self.cli_args.augmented or self.cli_args.augment_scale:
            self.augmentation_dict['scale'] = 0.2
        if self.cli_args.augmented or self.cli_args.augment_rotate:
            self.augmentation_dict['rotate'] = True
        if self.cli_args.augmented or self.cli_args.augment_noise:
            self.augmentation_dict['noise'] = 25.0

        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.model = self.initModel()
        self.optimizer = self.initOptimizer()


    def initModel(self):
        model = LunaModel()
        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1:
                model = nn.DataParallel(model)
            model = model.to(self.device)
        return model

    def initOptimizer(self):
        return SGD(self.model.parameters(), lr=0.001, momentum=0.99)
        # return Adam(self.model.parameters())

    def initTrainDl(self):
        train_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=False,
            ratio_int=int(self.cli_args.balanced),
            augmentation_dict=self.augmentation_dict,
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        train_dl = DataLoader(
            train_ds,
            batch_size=batch_size,
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda,
        )

        return train_dl

    def initValDl(self):
        val_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=True,
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        val_dl = DataLoader(
            val_ds,
            batch_size=batch_size,
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda,
        )

        return val_dl

    def initTensorboardWriters(self):
        if self.trn_writer is None:
            log_dir = os.path.join('runs', self.cli_args.tb_prefix, self.time_str)

            self.trn_writer = SummaryWriter(
                log_dir=log_dir + '-trn_cls-' + self.cli_args.comment)
            self.val_writer = SummaryWriter(
                log_dir=log_dir + '-val_cls-' + self.cli_args.comment)


    def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

## 11.3 사전 훈련 설정과 초기화

### 11.3.1 모델과 옵티마이저 초기화

In [ ]:
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.model = self.initModel()
        self.optimizer = self.initOptimizer()

    def initModel(self):
        model = LunaModel()
        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1:  # 복수 개의 GPU를 탐지
                model = nn.DataParallel(model)  # 모델을 래핑
            model = model.to(self.device)  # GPU에 모델 파라미터 전달
        return model


    def initOptimizer(self):
        return SGD(self.model.parameters(), lr=0.001, momentum=0.99)
        # return Adam(self.model.parameters())

### 11.3.2 데이터 로더의 관리와 데이터 공급

In [ ]:
    def initTrainDl(self):
        train_ds = LunaDataset(  # 커스텀 데이터셋
            val_stride=10,
            isValSet_bool=False,
            ratio_int=int(self.cli_args.balanced),
            augmentation_dict=self.augmentation_dict,
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        train_dl = DataLoader(  # 바로 사용하면 되는 클래스
            train_ds,
            batch_size=batch_size,  # 알아서 배치로 나뉜다.
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda,  # 고정된 메모리 영역이 GPU 쪽으로 빠르게 전송된다.
        )

        return train_dl
    

    def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl()  # 검증 데이터 로더는 훈련 데이터 로더와 매우 유사하다.

## 11.4 첫 번째 경로 신경망 설계

### 11.4.1 핵심 컨볼루션

In [46]:
import math

from torch import nn as nn

from util.logconf import logging

log = logging.getLogger(__name__)
# log.setLevel(logging.WARN)
# log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)


class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu2 = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool3d(2, 2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)


### 11.4.2 전체 모델

In [47]:
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1)

        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)

        self.head_linear = nn.Linear(1152, 2)
        self.head_softmax = nn.Softmax(dim=1)

        self._init_weights()



> 복잡한 문제: 컨볼루션을 선형으로 변환하기



In [48]:
    def forward(self, input_batch):
        bn_output = self.tail_batchnorm(input_batch)

        block_out = self.block1(bn_output)
        block_out = self.block2(block_out)
        block_out = self.block3(block_out)
        block_out = self.block4(block_out)

        conv_flat = block_out.view(
            block_out.size(0),
            -1,
        )
        linear_output = self.head_linear(conv_flat)

        return linear_output, self.head_softmax(linear_output)



> 초기화



In [ ]:
    def _init_weights(self):
        for m in self.modules():
            if type(m) in {
                nn.Linear,
                nn.Conv3d,
                nn.Conv2d,
                nn.ConvTranspose2d,
                nn.ConvTranspose3d,
            }:
                nn.init.kaiming_normal_(
                    m.weight.data, a=0, mode='fan_out', nonlinearity='relu',
                )
                if m.bias is not None:
                    fan_in, fan_out = \
                        nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)

## 11.5 모델 훈련과 검증

In [ ]:
def main(self):
    for epoch_ndx in range(1, self.cli_args.epochs + 1):
        trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
            self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)


    def doTraining(self, epoch_ndx, train_dl):
        self.model.train()
        train_dl.dataset.shuffleSamples()
        trnMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(train_dl.dataset),
            device=self.device,
        )

        batch_iter = enumerateWithEstimate(
            train_dl,
            "E{} Training".format(epoch_ndx),
            start_ndx=train_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.optimizer.zero_grad()

            loss_var = self.computeBatchLoss(
                batch_ndx,
                batch_tup,
                train_dl.batch_size,
                trnMetrics_g,
            )

            loss_var.backward()
            self.optimizer.step()

        self.totalTrainingSamples_count += len(train_dl.dataset)

        return trnMetrics_g.to('cpu')

### 11.5.1 computeBatchLoss 함수

In [ ]:
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
    input_t, label_t, _series_list, _center_list = batch_tup

    input_g = input_t.to(self.device, non_blocking=True)
    label_g = label_t.to(self.device, non_blocking=True)

    logits_g, probability_g = self.model(input_g)

    loss_func = nn.CrossEntropyLoss(reduction='none')
    loss_g = loss_func(
        logits_g,
        label_g[:,1],
    )
    start_ndx = batch_ndx * batch_size
    end_ndx = start_ndx + label_t.size(0)

    metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = label_g[:,1]
    metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = probability_g[:,1]
    metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = loss_g

    return loss_g.mean()

In [ ]:
METRICS_LABEL_NDX=0
METRICS_PRED_NDX=1
METRICS_LOSS_NDX=2
METRICS_SIZE = 3


    def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
        input_t, label_t, _series_list, _center_list = batch_tup

        input_g = input_t.to(self.device, non_blocking=True)
        label_g = label_t.to(self.device, non_blocking=True)

        logits_g, probability_g = self.model(input_g)

        loss_func = nn.CrossEntropyLoss(reduction='none')
        loss_g = loss_func(
            logits_g,
            label_g[:,1],
        )
        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + label_t.size(0)

        metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = label_g[:,1]
        metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = probability_g[:,1]
        metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = loss_g

        return loss_g.mean()  # 전체 배치에 대한 손실값이다.

### 11.5.2 훈련 때와 유사한 검증 루프

In [ ]:
def main(self):
    for epoch_ndx in range(1, self.cli_args.epochs + 1):
        valMetrics_t = self.doValidation(epoch_ndx, val_dl)
        
    self.logMetrics(epoch_ndx, 'val', valMetrics_t)


def doValidation(self, epoch_ndx, val_dl):
    with torch.no_grad():
        self.model.eval()
        valMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(val_dl.dataset),
            device=self.device,
        )

        batch_iter = enumerateWithEstimate(
            val_dl,
            "E{} Validation ".format(epoch_ndx),
                start_ndx=val_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.computeBatchLoss(
                batch_ndx,
                batch_tup,
                val_dl.batch_size,
                valMetrics_g,
            )

    return valMetrics_g.to('cpu')